# Imports

The imports are necessary for all three stages below, but each stage should be able to be run given that the previous stage has been completed.

In [1]:
import pandas as pd
import numpy as np

import random
import json
import os
import os.path as osp
import datetime

from youtube_dl import YoutubeDL

from movie_data_scraper import TMDB_Scraper

from google_auth_oauthlib.flow import InstalledAppFlow
from apiclient.discovery import build
from apiclient.http import MediaFileUpload
from apiclient import errors

yt_url_prefix = 'https://www.youtube.com/watch?v='
output_folder = 'casual_videos/'
tmdb_scraper = TMDB_Scraper(api_key="abd17a9f250807b76ebbfa9997ca6ade")

# 3) Download Casual Videos to Local Drive

In [5]:
trailers = ['']

In [9]:
def download_trailers(trailers, dataset_dir, ydl_opts={}, verbose=False):
    ydl_opts['outtmpl'] = dataset_dir + '%(id)s.%(ext)s'
    downloaded = os.listdir(dataset_dir)
    downloaded = [d.split('.')[0] for d in downloaded]
    downloaded = set(downloaded)
    with YoutubeDL(ydl_opts) as ydl:
        for t in trailers:
            try:
                # Helpful Constants
                yt_link = t

                # Download Video
                print(f"Downloading {t}")
                trailer_yt_info = ydl.extract_info(yt_link, download=True)
            except Exception as e:
                print(f"Failed with exception")
                print(e)
            print()

#### High(est) Quality

In [13]:
download_trailers("../data/", {'format': 'bestvideo'}, verbose=True)

remaining trailers (len(19)) {'uJQZdk5jMEg', 'TG5_hQaUUHs', 'TuLX50_5UAI', 'JM-0Ywc7wNY', '5DEdq57E5ls', 'by2yMNLmCDs', 'AyAg2gu6z0I', 'iuAzsEKD_vI', 'Biigna1PMOw', 'p4-6qJzeb3A', 't0O2XsTxe3o', 'XNPAd4if3gY', 'FlNGXOs_vSM', 'VvHquOz-lDU', 'GrqpKEOF8uw', '5TdZQDbRNKU', 'YMmrKORgNFc', 'FeBa2s4ldU4', '1x4OmSgR714'}
[youtube] uJQZdk5jMEg: Downloading webpage
[youtube] uJQZdk5jMEg: Downloading video info webpage


ERROR: This video is unavailable.
Sorry about that.


Failed with exception
ERROR: This video is unavailable.
Sorry about that.

[youtube] TG5_hQaUUHs: Downloading webpage
[youtube] TG5_hQaUUHs: Downloading video info webpage


ERROR: This video is unavailable.
Sorry about that.


Failed with exception
ERROR: This video is unavailable.
Sorry about that.

[youtube] TuLX50_5UAI: Downloading webpage
[youtube] TuLX50_5UAI: Downloading video info webpage


...

ERROR: This video is unavailable.
Sorry about that.


Failed with exception
ERROR: This video is unavailable.
Sorry about that.

[youtube] by2yMNLmCDs: Downloading webpage
[youtube] by2yMNLmCDs: Downloading video info webpage


ERROR: This video is no longer available because the YouTube account associated with this video has been terminated.
Sorry about that.


Failed with exception
ERROR: This video is no longer available because the YouTube account associated with this video has been terminated.
Sorry about that.

[youtube] AyAg2gu6z0I: Downloading webpage


KeyboardInterrupt: 

# 4) Upload Movie Trailers to Google Drive (optional)

In [2]:
# Uncomment to install dependencies
#!pip install pandas
#!pip install youtube_dl
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
# GOOGLE DRIVE API CONSTANTS
flow = InstalledAppFlow.from_client_secrets_file('credentials.json', ['https://www.googleapis.com/auth/drive'])
creds = flow.run_local_server(port=0)
folder_id = '1UnDIe4VHMM8bZzIfKEG8NJdQGnNrrxEG'
drive_api = build('drive', 'v3', credentials=creds)

In [19]:
# Helpful function
def get_file_list_from_folder(service, folder_id):
    """Print files belonging to a folder.

    Args:
    service: Drive API service instance.
    folder_id: ID of the folder to print files from.
    """
    kwargs = {
        "q": "'{}' in parents".format(folder_id)
    }
    request = service.files().list(**kwargs)
    files = []
    while request is not None:
        response = request.execute()
        # Do stuff with response['files']
        files.extend(response['files'])
        request = service.files().list_next(request, response)
    return files

In [24]:
trailers = pd.read_csv(osp.join(output_folder, "trailers.csv"), index_col=0)
trailers['movie_title'].unique().size

1171

In [ ]:
ydl_opts = {
    'outtmpl': output_folder + '%(id)s.%(ext)s',
    'format': 'bestvideo'
}

uploaded = get_file_list_from_folder(drive_api, folder_id)
#print(uploaded)
uploaded = [d['name'].split('.')[0] for d in uploaded]
duplicates = pd.Series(uploaded).value_counts()
uploaded = set(uploaded)
#print(duplicates)
#print(f'files already uploaded {uploaded}')

all_to_download = set(trailers['youtube_key'])
#print(f'files already uploaded & requested for upload{uploaded & all_to_download}')
remaining_trailers = all_to_download - uploaded
print(f'remaining trailers (len({len(remaining_trailers)})) {remaining_trailers}')

In [ ]:
with YoutubeDL(ydl_opts) as ydl:
    for t in remaining_trailers:
        try:
            # Helpful Constants
            yt_link = yt_url_prefix + t

            # Download Video
            print(f"Downloading {t}")
            trailer_yt_info = ydl.extract_info(yt_link, download=True)
            
            # Determine File Name
            file_name = [f for f in os.listdir(output_folder) if t in f][0]
            print(file_name)

            # Upload to Drive
            print(f"Uploading file {file_name}...")
            body = {'name': file_name, 'parents': [folder_id]}
            media = MediaFileUpload(output_folder + file_name)
            fiahl = drive_api.files().create(body=body, media_body=media).execute()
            print(f"Created file '{fiahl.get('name')}' id '{fiahl.get('id')}'.")

            # Delete video from hard drive
            os.remove(output_folder + file_name)
            print(f"Removed {file_name}")
            print("Success!")
        except Exception as e:
            print(f"Failed with exception")
            print(e)
        print()